In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as sco

from polyline import Polyline

from curvegen import Curve2D
from egoaxis import rot_axis_90deg_cc

### Test distance to curve vertex gradient

In [ ]:
def get_gradient(dists_grad, proj_to, vidx):
    """ Select proper analytic gradients depending on the projection region. """
    grad = np.zeros((len(dists_grad), 2), dtype=np.float)
    
    # Projects on segment, choose first / second vertex to get middle one
    for i, gi in enumerate(dists_grad):
        proj_id = proj_to[i]
        if proj_id < 0:
            # Projected on segment, select first or second gradient
            if proj_id == -vidx:
                # First segment, need gradient from second vertex
                grad[i] = dists_grad[i][1]
            if proj_id == -vidx - 1:
                # Second segment, need gradient from first vertex
                grad[i] = dists_grad[i][0]
        else:
            # If projected on proper vertex, only first gradient is defined
            if proj_id == vidx:
                grad[i] = dists_grad[i][0]
            else:
                # For all other vertices, the second gradient is set to zero
                grad[i] = dists_grad[i][1] 
    return grad

def plot_it(which, data_pt, pos,
            dists, ana_grad, num_grad, proj_to, proj_vecs, pl):
    """ which can be 'x' or 'y'. """
    if pl.nvertices != 3:
        raise ValueError("Plot function works only with 3 vertices")
    
    if which not in ["x", "y"]:
        raise ValueError("which can be 'x' or 'y'.")
        
    data_pt = np.atleast_2d(data_pt)
    if data_pt.shape != (1, 2):
        raise ValueError(
            "Plot function works only with single data point with shape (1, 2)")
    
    axis = 0 if which == "x" else 1
    _verts = pl.vertices  # Store for restoring them later
    fig, (axl, axr) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Analytic gradient
    axl.plot(pos[:, axis], dists, c="C0")
    axl.plot(pos[:, axis], ana_grad[:, axis], c="C1", label="ana")
    axl.plot(pos[:, axis], num_grad, c="k", ls=":", label="num")
    axl.plot(pos[:, axis], proj_to, c="C7", ls="--", label="proj_to")
    axl.set_xlabel(which)
    axl.legend()
    axl.grid()

    # Plot changing vertices and projection vectors for each position
    idx = np.concatenate((np.arange(len(pos))[::50], [len(pos) - 1]))
    for i in idx:
        c = "C0"
        if i == 0:
            c = "C2"  # Start vertex green
        elif i == (len(pos) - 1):
            c = "C3"  # End vertex red
        pl.replace_vertex(1, [pos[i, 0], pos[i, 1]])
        axr.plot(pl.xcoords, pl.ycoords, marker="o", c=c)
        # Projection vec
        axr.plot([data_pt[0, 0], data_pt[0, 0] + proj_vecs[i, 0]],
                 [data_pt[0, 1], data_pt[0, 1] + proj_vecs[i, 1]], c="C1")

    axr.plot(data_pt[0, 0], data_pt[0, 1], c="C1", marker="o")
    axr.set_xlabel(which)
    axr.set_aspect("equal")

    pl = Polyline(_verts)
    
    plt.show()

In [ ]:
pl = Polyline([[-0.5, -2], [0, 1], [-0.5, 2]])
vidx = 1  # Movable vertex ID

data_pts = np.array([[-2, 0.5]])

# Test x gradient, varied vertex is v0 of segment
npts = 500
x = np.linspace(-1, 3, npts)
y = np.array(len(x) * [0.5])

dists, dists_grad, proj_to, proj_vecs = [], [], [], []

for xi, yi in zip(x, y):
    pl.replace_vertex(vidx, [xi, yi])
    _d, _dg, _pto, _pvec = pl.get_dist_to_line(data_pts)

    dists.append(_d)
    dists_grad.append(_dg)
    proj_to.append(_pto)
    proj_vecs.append(_pvec)

dists = np.squeeze(dists)
dists_grad = np.squeeze(dists_grad)
proj_to = np.squeeze(proj_to)
proj_vecs = np.squeeze(proj_vecs)
# Directional gradient along x for middle vertex
num_grad = np.gradient(dists, x)
ana_grad = get_gradient(dists_grad, proj_to, vidx)
            
# Plot it
pos = np.vstack((x, y)).T
plot_it("x", data_pts, pos, dists, ana_grad, num_grad, proj_to, proj_vecs, pl)

In [ ]:
pl = Polyline([[-0.5, -2], [0, 1], [-0.5, 2]])
vidx = 1  # Movable vertex ID

data_pts = np.array([[-2, -0.5]])

# Test x gradient, varied vertex is v1 of segment
npts = 500
x = np.linspace(-1, 3, npts)
y = np.array(len(x) * [-0.5])

dists, dists_grad, proj_to, proj_vecs = [], [], [], []

for xi, yi in zip(x, y):
    pl.replace_vertex(vidx, [xi, yi])
    _d, _dg, _pto, _pvec = pl.get_dist_to_line(data_pts)

    dists.append(_d)
    dists_grad.append(_dg)
    proj_to.append(_pto)
    proj_vecs.append(_pvec)

dists = np.squeeze(dists)
dists_grad = np.squeeze(dists_grad)
proj_to = np.squeeze(proj_to)
proj_vecs = np.squeeze(proj_vecs)
# Directional gradient along x for middle vertex
num_grad = np.gradient(dists, x)
ana_grad = get_gradient(dists_grad, proj_to, vidx)
            
# Plot it
pos = np.vstack((x, y)).T
plot_it("x", data_pts, pos, dists, ana_grad, num_grad, proj_to, proj_vecs, pl)

In [ ]:
pl = Polyline([[-1, 1.5], [0, 0], [3, 1.5]])
vidx = 1  # Movable vertex ID

data_pts = np.array([[0.5, 3]])

# Test y gradient, varied vertex is v1 of segment
npts = 500
y = np.linspace(-2, 2, npts)
x = np.array(len(y) * [0.5])

dists, dists_grad, proj_to, proj_vecs = [], [], [], []

for xi, yi in zip(x, y):
    pl.replace_vertex(vidx, [xi, yi])
    _d, _dg, _pto, _pvec = pl.get_dist_to_line(data_pts)

    dists.append(_d)
    dists_grad.append(_dg)
    proj_to.append(_pto)
    proj_vecs.append(_pvec)

dists = np.squeeze(dists)
dists_grad = np.squeeze(dists_grad)
proj_to = np.squeeze(proj_to)
proj_vecs = np.squeeze(proj_vecs)
# Directional gradient along y for middle vertex
num_grad = np.gradient(dists, y)
ana_grad = get_gradient(dists_grad, proj_to, vidx)
            
# Plot it
pos = np.vstack((x, y)).T
plot_it("y", data_pts, pos, dists, ana_grad, num_grad, proj_to, proj_vecs, pl)

In [ ]:
pl = Polyline([[-1, 1.5], [0, 0], [3, 1.5]])
vidx = 1  # Movable vertex ID

data_pts = np.array([[1.5, 3]])

# Test y gradient, varied vertex is v0 of segment
npts = 500
y = np.linspace(-2, 2, npts)
x = np.array(len(y) * [1.5])

dists, dists_grad, proj_to, proj_vecs = [], [], [], []

for xi, yi in zip(x, y):
    pl.replace_vertex(vidx, [xi, yi])
    _d, _dg, _pto, _pvec = pl.get_dist_to_line(data_pts)

    dists.append(_d)
    dists_grad.append(_dg)
    proj_to.append(_pto)
    proj_vecs.append(_pvec)

dists = np.squeeze(dists)
dists_grad = np.squeeze(dists_grad)
proj_to = np.squeeze(proj_to)
proj_vecs = np.squeeze(proj_vecs)
# Directional gradient along y for middle vertex
num_grad = np.gradient(dists, y)
ana_grad = get_gradient(dists_grad, proj_to, vidx)
            
# Plot it
pos = np.vstack((x, y)).T
plot_it("y", data_pts, pos, dists, ana_grad, num_grad, proj_to, proj_vecs, pl)

## Module

In [ ]:
# Note:
# To make it match with the previous measurement, init with the last point
# of the previous measurement and keep it fixed.
# Use the largest principal component vector, project points, keep last point
# fixed an rotate line around this point, so the first point matches the
# reference point from the last measurement.

def proj_points(pts, vec):
    """
    Projects points with shape (len(vec), N) along direction of vec (1d-array).
    """
    vec = np.asarray(vec)
    pts = np.atleast_2d(pts)
    
    unit = vec / np.linalg.norm(vec)
    proj_len = np.matmul(pts.T, vec)

    return proj_len * vec.reshape(len(vec), 1), proj_len

def poly_principal_curve(data, niter="auto"):
    """
    Computes a principal curve for the given data points (shape (2, n)).
    Source
    ------
    Kegl: Learning and Design of Principal Curves
        (https://www.lri.fr/~kegl/research/PDFs/KeKrLiZe00.pdf)
    """
    data = np.atleast_2d(data)
    if len(data.shape) != 2 or data.shape[0] != 2:
        raise ValueError("'data' must have shape (2, nPoints).")
    npoints = data.shape[1]
    
    # Center data mean at (0, 0)
    mean = np.mean(data, axis=1).reshape(2, 1)
    data -= mean
    
    # Following the algorithm in chapter 3 of the paper
    
    # Initialization: Use first principal value of data as projection line d and
    # pick the line length L as the distance of min, max points projected on d
    # Get direction and value of largest eigenvalue from cov(data)
    cov = np.cov(data)
    eigvals, eigvecs = np.linalg.eigh(cov)
    eigvec_max = eigvecs[np.argmax(eigvals)]
    # Project along largest eigenvector and determine line length by finding
    # the outermost projected points along the projection line
    proj_vec, proj_len = proj_points(data, eigvec_max)
    idx = np.argsort(proj_len)
    # Keep the original order of data points intact for the projection
    idx_min, idx_max = ((idx[0], idx[-1]) if
                        idx[0] < idx[-1] else (idx[-1], idx[0]))
    dmin, dmax = data[:, idx_min], data[:, idx_max]
    pl = Polyline([dmin, dmax])
    
    # Data radius used for scaling in the penalty term
    data_r = np.amax(np.linalg.norm(data, axis=0))**2
    data_r2 = data_r**2
    
    def loss(idx, pl, lambda_p):
        """
        Return loss term for given vertex idx in polyline pl.
        lambda_p > 0 defines the strength of the penalty term.
        """
        # Curvature penalty
        def pi(i):
            cos_ang, cos_ang_grad = pl.get_cos_angle_at_vertex(i)
            func = data_r2 * (1. + cos_ang)
            # Use chain rule:
            # dx (r2 * (1 + cos(ang))) = r2 * -sin(1 + cos(ang)) * dx cos(ang)
            grad = data_r2 * -np.sin(1. + func) * cos_ang_grad
            return func, grad
        # Todo: Gradient of segment length when vertex i changes
        def mu_plus(i):
            """ Segment length depending on first vertex in segment """
            fun = pl.get_segment_length(i)  # Segment (i, i+1)
            # Vertex i first -> Gradient of segment (i, i+1), vi = first vertex
            grad = pl.get_segment_length_grad(i, which="first")
            return func, grad
        
        def mu_minus(i):
            """ Segment length depending on last vertex in segment """
            fun = pl.get_segment_length(i - 1)  # Segment (i-1, i)
            # Vertex i last -> Gradient of segment (i-1, i), vi = last vertex
            grad = pl.get_segment_length_grad(i - 1, which="last")
            return func, grad

        if idx == 0:
            mup, mupg = mu_plus(idx)
            pi1, pi1g = pi(idx + 1)
            penalty, penalty_grad = mup + pi1, mupg + pi1g
        elif idx == 1:
            mum, mumg = mu_minus(idx)
            pi0, pi0g = pi(idx)
            pi1, pi1g = pi(idx + 1)
            penalty, penalty_grad = mum + pi0 + pi1, mumg + pi0g + pi1g
        elif idx == pl.nvertices - 2:
            mup, mupg = mu_plus(idx)
            pi0, pi0g = pi(idx)
            pi1n, pi1ng = pi(idx - 1)
            penalty, penalty_grad = pi1n + pi0 + mup, pi1ng + pi0g + mupg
        elif idx == pl.nvertices - 1:
            mum, mumg = mu_minus(idx)
            pi1n, pi1ng = pi(idx - 1)
            penalty, penalty_grad = pi1n + mum, pi1ng + mumg
        else:
            pi0, pi0g = pi(idx)
            pi1, pi1g = pi(idx + 1)
            pi1n, pi1ng = pi(idx - 1)
            penalty, penalty_grad = pi1n + pi0 + pi1, pi1ng + pi0g + pi1g
        
        # Local distance loss
        # Project and segment points for local losses
        _, dists, partitions = pl.get_dist_to_line(data)
        
        def sigma_plus(i):
            """
            Sum of squared distance of data points projecting to segment
            Si -> (vi, v_(i+1)) of current polyline to segment Si.
            """
            # Segment (vi, v_(i+1)) is encoded with -(i+1) ((v0, v1) -> S_(-1))
            seg_idx = -1 * (i + 1)
            mask = partitions == seg_idx
            return np.sum(dists[mask]**2)
    
        def sigma_minus(i):
            """ Same for segment Si -> (v-(i-1), vi) """
            # Segment (vi, v_(i+1)) is encoded with -(i+1) ((v0, v1) -> S_(-1))
            seg_idx = -1 * i  # Is never called for i=0
            mask = partitions == seg_idx
            return np.sum(dists[mask]**2)
        
        def nu(i):
            """
            Sum of squared distance of data points projecting to vertex vi.
            """
            mask = partitions == i
            return np.sum(dists[mask]**2)
        
        if idx == 0:
            local_dist_loss = nu(idx) + sigma_plus(idx)
        elif idx == pl.nvertices - 1:
            local_dist_loss = sigma_minus(idx) + nu(idx)
        else:
            local_dist_loss = sigma_minus(idx) + nu(idx) + sigma_plus(idx)
        
        loss = (local_dist_loss / float(npoints) +
                lambda_p * penalty / pl.nvertices)
        
        # Gradient information, dloss / dvi
        
    
    def avrg_sqrd_dist_to_polyline(pl):
        """
        Returns 1 / n * sum_i(norm(delta(pl, di))**2), which is the mean squared
        distance of all data points di to the current polyline pl.
        """
        _, dists, _ = pl.get_dist_to_line(data)
        return np.sum(dists**2) / float(npoints)
        
    def converged(pl, beta=0.3):
        """ Returns True, if given polyline reaches convergence criteria """
        return pl.nsegments > (beta * npoints**(1. / 3.) /
                               np.sqrt(avrg_sqrd_dist_to_polyline(pl)) * data_r)
    
    def step_converged(pl):
        """ Returns true if the current vertex step is converged """
        return False  # TODO
    
    # Iteration: Add vertex each iter until converged or max vertices reached
    # Empirical from paper, at least 2
    LAM_P_STAR = 0.13  # Curvature penalty strength scaling constant
    while True:
        # Vertex optimization:
        # Optimize each existing vertex while fixing the others until the
        # polyline with k current vertices is locally closest to all points
        lambda_p = (LAM_P_STAR * pl.nsegments * npoints**(-1. / 3.) *
                    np.sqrt(avrg_sqrd_dist_to_polyline(pl)) / data_r)
        while not step_converged(pl):
            for j, vert in pl.vertices:
                pass
            break  # TODO: Remove when step_converged is implemented

        if converged(pl):
            break  # Break early if converged
            
        # Add vertex at longest segment and run new optimization
        idx = np.argmax(pl.segment_lengths)
        pl.insert_vertex_between(idx + 1, scale=0.5)
    
    # Move back to actual mean
    pl.translate(mean)
    return pl

## Testplots

### Angle gradients

In [ ]:
# Middle index is moved on a grid to test dx and dy components against the
# numerical ones from a finely sampled grid
# Use a little offset to make sure vertices don't overlap with sample points
pl = Polyline([[-1.03, -1.03], [0, 0], [1.03, 1.03]])
idx = 1  # Movable vertex id (middle vertex to obtain mutliple angles)

x_pos = np.linspace(-2, 2, 501)  # x-coords for movable middle vertex
y_pos = np.linspace(-2, 2, 501)  # y-coords for movable middle vertex
xx, yy = np.meshgrid(x_pos, y_pos)  # axis 1 varies x, axis 0 varies y

cos_angles, cos_angles_grad_x, cos_angles_grad_y = [], [], []
for xi, yi in zip(xx.flatten(), yy.flatten()):
    pl.replace_vertex(idx, [xi, yi])
    cos_ang, cos_ang_grad = pl.get_cos_angle_at_vertex(idx)
    cos_angles.append(cos_ang)
    cos_angles_grad_x.append(cos_ang_grad[0])
    cos_angles_grad_y.append(cos_ang_grad[1])

cos_angles = np.array(cos_angles).reshape(xx.shape)
cos_angles_grad_x = np.array(cos_angles_grad_x).reshape(xx.shape)
# Transposed matrix has y gradients for each x per row, so: y_pos, grad_y[idx]
cos_angles_grad_y = np.array(cos_angles_grad_y).reshape(xx.shape).T
num_grad_x = np.gradient(cos_angles, x_pos, axis=1)  # axis 1 varies x (-2, 2)
# Somehow using axis = 0 is giving the wrong gradients but the same ones as
# analytical ones. Transposing first and using axis=1 should acutally be the
# same but in this case somehow isn't
num_grad_y = np.gradient(cos_angles.T, y_pos, axis=1)  # axis 0 varies y (-2, 2)

In [ ]:
idxs = []
for idx in range(len(cos_angles)):
    # Num grad and ana grad are close enough?
    if not (np.allclose(num_grad_x[idx], cos_angles_grad_x[idx], atol=1e-2) and
            np.allclose(num_grad_y[idx], cos_angles_grad_y[idx], atol=1e-2)):
        # Some are quite steep so the numerical error is high in any case
        print(idx)
        print("  Max dev x: ", np.amax(num_grad_x[idx] - cos_angles_grad_x[idx]))
        print("  Max dev y: ", np.amax(num_grad_y[idx] - cos_angles_grad_y[idx]))
        idxs.append(idx)

if idxs:
    ", ".join(map(str, idxs))

In [ ]:
# Plot some of those with high deviation
# for idx in list(range(len(x_pos))[::10]) + [25]:
for idx in idxs[::10]:
    fig, (axl, axr) = plt.subplots(1, 2, figsize=(12, 5))
    
    axl.plot(x_pos, cos_angles[idx], c="C0", label="x")
    axl.plot(x_pos, cos_angles_grad_x[idx], label="dx", c="C2")
    axl.plot(x_pos, num_grad_x[idx], label="dx (num)", ls=":", c="k")
    axl.set_title("({}, {}) -> (x, {:.2f}) -> ({}, {})".format(
        pl.vertices[0, 0], pl.vertices[0, 1],
        y_pos[idx],
        pl.vertices[2, 0], pl.vertices[2, 1],
    ))
    axl.set_xlabel("x")

    # Plot y-vals on same axis
    axr.plot(y_pos, cos_angles[:, idx], c="C1", label="y")
    axr.plot(y_pos, cos_angles_grad_y[idx], label="dy", c="C3")
    axr.plot(y_pos, num_grad_y[idx], label="dy (num)", ls="--", c="k")
    axr.set_title("({}, {}) -> ({:.2f}, y) -> ({}, {})".format(
        pl.vertices[0, 0], pl.vertices[0, 1],
        x_pos[idx],
        pl.vertices[2, 0], pl.vertices[2, 1],
    ))
    axr.set_xlabel("y")

    for axi in (axl, axr):
        axi.axvline(0, 0, 1, c="k", lw=1)
        axi.axhline(0, 0, 1, c="k", lw=1)
        axi.legend()
        axi.grid()
        
    fig.suptitle("idx = {}".format(idx))
    plt.show()

### Curve generator

In [ ]:
curve = Curve2D(curvature=-3., curve_rel_start=0.6)
x = np.linspace(0, 10, 50)
y, dy = curve(x)
x_sam = np.linspace(1, 10, 20)
x_sam, y_sam = curve.sample(x_sam, stddev=0.4, seed=2)

fig, ax = plt.subplots(1, 1)
ax.plot(x, y)
ax.plot(x_sam, y_sam, marker="o", ls="-")

ax.set_xlim(0, 1.1 * max(np.amax(x_sam), np.amax(x)))
ax.set_ylim(-5, 5)

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.plot(0, 0, marker=10, ms=10, ls="none", c="k")

ax = rot_axis_90deg_cc(ax)
ax.grid()
ax.axvline(0, 0, 1, c="k", lw=1, zorder=1)
ax.set_axisbelow(True)

plt.show()

### Distance to polyline

In [ ]:
pl = Polyline([[1, 3], [3, 5], [6, 2], [2, 2]])
pts = np.array(
    [[1, 2], [1.5, 5.5], [3, 6], [3.5, 3.5], [5.5, 3.5], [4, 1], [2, 2.5]])
dists, dists_grad, proj_to, proj_vecs = pl.get_dist_to_line(pts)

In [ ]:
plt.plot(pl.xcoords, pl.ycoords, ls="-", marker="o", c="C0")
plt.plot(pts[:, 0], pts[:, 1], marker="x", lw=4, c="k", ls="none")

for pti, pvi in zip(pts, proj_vecs):
    plt.arrow(pti[0], pti[1], pvi[0], pvi[1])
    
print(proj_to)
print(dists)

plt.xlim(0, None)
plt.ylim(0, None)
plt.gca().set_aspect("equal")
plt.show()

### Gradient of distance to polyline with respect to point change

#### Compare to numerical gradient

In [ ]:
pl = Polyline([[0, 0], [1, 0.8], [1.1, 1.9]])
npts = 500

# Test x gradient
ys = [-1, 2]
x = np.linspace(-1, 2, npts)

for yi in ys:
    y = yi + np.zeros_like(x)
    pts = np.vstack((x, y)).T

    dists, _, _, proj_vecs = pl.get_dist_to_line(pts)

    # Directional gradient along x
    num_grad = np.gradient(dists, x)
    # Gradient with respet to point is normed projection vector
    grad = -proj_vecs / np.linalg.norm(
        proj_vecs, axis=1).reshape(len(proj_vecs), 1)
    grad = grad[:, 0]
    
    if not np.allclose(num_grad, grad, atol=1e-2):
        dgrad = np.abs(num_grad - grad)
        idx = np.argmax(dgrad)
        print("yi : ", yi, " max dev : ", np.amax(dgrad),
              " at ", x[idx], " (index : ", idx, ")")

    fig, (axl, axr) = plt.subplots(1, 2, figsize=(12, 5))

    axl.plot(pl.xcoords, pl.ycoords, marker="o", c="C0")
    s = slice(None, None, 20)
    axl.plot(x[s], y[s], marker="o", ls="", c="C1")
    for i, (xi, yi) in enumerate(zip(x[s], y[s])):
        axl.plot([xi, xi + proj_vecs[s][i, 0]],
                 [yi, yi + proj_vecs[s][i, 1]],
                 c="C1")
        
    axr.plot(x, dists, c="C0")
    axr.plot(x, grad, c="C1")
    axr.plot(x, num_grad, c="k", ls=":")
    axr.set_xlabel("x")
    
    axl.set_aspect("equal")
    plt.show()
    
# Test y gradient
xs = [-0.5, 2]
y = np.linspace(-1, 1.5, npts)

for xi in xs:
    x = xi + np.zeros_like(y)
    pts = np.vstack((x, y)).T

    dists, dists_grad, _, proj_vecs = pl.get_dist_to_line(pts)

    # Directional gradient along x
    num_grad = np.gradient(dists, y)
    # Gradient with respet to point is normed projection vector
    grad = -proj_vecs / np.linalg.norm(
        proj_vecs, axis=1).reshape(len(proj_vecs), 1)
    grad = grad[:, 1]

    if not np.allclose(num_grad, grad, atol=1e-2):
        dgrad = np.abs(num_grad - grad)
        idx = np.argmax(dgrad)
        print("xi : ", xi, " max dev : ", np.amax(dgrad),
              " at ", y[idx], " (index : ", idx, ")")
    
    fig, (axl, axr) = plt.subplots(1, 2, figsize=(12, 5))

    axl.plot(pl.xcoords, pl.ycoords, marker="o", c="C0")
    s = slice(None, None, 20)
    axl.plot(x[s], y[s], marker="o", ls="", c="C1")
    for i, (xi, yi) in enumerate(zip(x[s], y[s])):
        axl.plot([xi, xi + proj_vecs[s][i, 0]],
                 [yi, yi + proj_vecs[s][i, 1]],
                 c="C1")

    axr.plot(y, dists, c="C0")
    axr.plot(y, grad, c="C1")
    axr.plot(y, num_grad, c="k", ls=":")
    axr.set_xlabel("y")

    axl.set_aspect("equal")
    plt.show()

#### Vector field of gradients

In [ ]:
# Sample a grid of points and draw the line distance gradient vector field
vertices = np.array([[0.5, 1], [1, 0], [2, 0], [2, 2]])
pl = Polyline(vertices)

grid_pts = 1000
xmin, ymin = np.amin(vertices, axis=0)
xmax, ymax = np.amax(vertices, axis=0)
if np.isclose(ymin, ymax):
    ymin -= 0.5
    ymax += 0.5
if np.isclose(xmin, xmax):
    xmin -= 0.5
    xmax += 0.5
off = 1
x = np.linspace(xmin - off, xmax + off, grid_pts)
y = np.linspace(ymin - off, ymax + off, grid_pts)

XX, YY = np.meshgrid(x, y)
xx, yy = map(np.ravel, [XX, YY])
pts = np.vstack((xx, yy)).T

dists, _, proj_to, proj_vecs = pl.get_dist_to_line(pts)

# Gradients are negative normalized connection vectors
grads = np.zeros_like(proj_vecs)
valid = dists > 0
grads[valid] = -proj_vecs[valid] / np.linalg.norm(
    proj_vecs, axis=1)[valid].reshape(np.sum(valid), 1)

In [ ]:
# Plot vectorfield with polyline and segmentation
dx = grads[:, 0].reshape(grid_pts, grid_pts)
dy = grads[:, 1].reshape(grid_pts, grid_pts)

# Segmentation
zz = proj_to.reshape(grid_pts, grid_pts)
levels = np.arange(np.amin(zz), np.amax(zz) + 2, 1) - 0.5
colors = plt.cm.magma(np.linspace(0, 1, len(levels)))
_dx, _dy = x[1] - x[0], y[1] - y[0]
_x = np.r_[x, x[-1] + _dx] - _dx / 2.
_y = np.r_[y, y[-1] + _dy] - _dy / 2.
_XX, _YY = np.meshgrid(_x, _y)
plt.pcolormesh(x, y, zz, shading="flat", cmap="tab20c")

# Vectorfield
s = slice(None, None, 50)  # Plot only every 50th arrow
plt.quiver(XX[s, s], YY[s, s], dx[s, s], dy[s, s], color="w")

# Polyline
plt.plot(pl.xcoords, pl.ycoords, ls="-", marker="o", c="k")

plt.gca().set_aspect("equal")
plt.show()

### Project on eigenvector

In [ ]:
curve = Curve2D(curvature=-4., curve_rel_start=0.3)
x = np.linspace(0, 10, 50)
y, dy = curve(x)
x_sam = np.linspace(1, 10, 20)
x_sam, y_sam = curve.sample(x_sam, stddev=0.4, seed=2)

data = np.asarray([x_sam, y_sam])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 6))

# Truth and sampled points
ax.plot(x, y)
ax.plot(x_sam, y_sam, marker="o", ls="-")

ax.set_xlim(0, 1.1 * max(np.amax(x_sam), np.amax(x)))
ax.set_ylim(-5, 5)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.plot(0, 0, marker=10, ms=10, ls="none", c="k")

# Principal value line
eigvals, eigvecs = np.linalg.eigh(np.cov(data))
eigvec_max = eigvecs[np.argmax(eigvals)]
mean = np.mean(data, axis=1)
mx = np.r_[mean[0] - 10 * eigvec_max[0], mean[0] + 10 * eigvec_max[0]]
my = np.r_[mean[1] - 10 * eigvec_max[1], mean[1] + 10 * eigvec_max[1]]
ax.plot(mean[0], mean[1], ls="none", marker="x", ms=10, c="C3", mew=3, zorder=1)
ax.plot(mx, my, ls="--", c="C3", lw=2)

# Projected points
mean = np.mean(data, axis=1).reshape(2, 1)
proj_vec, proj_len = proj_points(data - mean, eigvec_max)
proj_vec += mean
for proj, pt in zip(proj_vec.T, data.T):
    dx = [proj[0], pt[0]]
    dy = [proj[1], pt[1]]
    ax.plot(dx, dy, lw=1, c="k")

# ax = rot_axis_90deg_cc(ax)
ax.grid()
ax.axvline(0, 0, 1, c="k", lw=1, zorder=1)
ax.axhline(0, 0, 1, c="k", lw=1, zorder=1)
ax.set_axisbelow(True)
ax.set_aspect("equal")

plt.show()